# Test Pivoting Merged KG Edges 
### This notebooks tests various approaches for pivoting data in `data/merged-kg_edges.tsv.gz`

In [1]:
import pandas as pds

## load dataframe using file merged-kg_edges.tsv.gz

In [16]:
df = pds.read_csv('data/merged-kg_edges.tsv.gz', sep='\t')
len(df)

424612

In [17]:
df.head() # peek at data

,id,subject,predicate,object,relation,provided_by
0,urn:uuid:875773d1-301d-46e4-bc42-1156d7cfaafa,CHEBI:51503,biolink:subclass_of,CHEBI:51494,rdfs:subClassOf,chebi
1,urn:uuid:6c7238d0-5836-4cfe-be27-63837059086b,CHEBI:26547,biolink:subclass_of,CHEBI:35315,rdfs:subClassOf,chebi
2,urn:uuid:f64ba0ef-1152-40b5-b245-bed32d57443f,CHEBI:51504,biolink:subclass_of,CHEBI:51493,rdfs:subClassOf,chebi
3,urn:uuid:eebedf42-5961-4208-aaae-44796c6574e6,CHEBI:26548,biolink:subclass_of,CHEBI:24400,rdfs:subClassOf,chebi
4,urn:uuid:a2ef1a16-b470-4b35-a6c9-0a2681b3fe2f,CHEBI:51505,biolink:subclass_of,CHEBI:51504,rdfs:subClassOf,chebi


## approach 1: create a reduced dataframe using subset of objects values
- To make visual inspection easier, get 5 unique object values.
- Filter `df` to include only rows with these objects, results in `reducedDf`.

In [4]:
objects = list(df['object'].unique())
len(objects)

47792

In [5]:
test_objects = objects[:5]
test_objects

['CHEBI:51494', 'CHEBI:35315', 'CHEBI:51493', 'CHEBI:24400', 'CHEBI:51504']

### build test dataframe by filtering rows by test_objects

In [6]:
reducedDf = df[df['object'].isin(test_objects)]
len(reducedDf)

644

In [7]:
reducedDf.head() # peek at data

,id,subject,predicate,object,relation,provided_by
0,urn:uuid:875773d1-301d-46e4-bc42-1156d7cfaafa,CHEBI:51503,biolink:subclass_of,CHEBI:51494,rdfs:subClassOf,chebi
1,urn:uuid:6c7238d0-5836-4cfe-be27-63837059086b,CHEBI:26547,biolink:subclass_of,CHEBI:35315,rdfs:subClassOf,chebi
2,urn:uuid:f64ba0ef-1152-40b5-b245-bed32d57443f,CHEBI:51504,biolink:subclass_of,CHEBI:51493,rdfs:subClassOf,chebi
3,urn:uuid:eebedf42-5961-4208-aaae-44796c6574e6,CHEBI:26548,biolink:subclass_of,CHEBI:24400,rdfs:subClassOf,chebi
4,urn:uuid:a2ef1a16-b470-4b35-a6c9-0a2681b3fe2f,CHEBI:51505,biolink:subclass_of,CHEBI:51504,rdfs:subClassOf,chebi


### pivot `reducedDf` using the `id` column as the index

In [8]:
reducedDf.head().pivot(index='id', columns='object', values='subject')

object,CHEBI:24400,CHEBI:35315,CHEBI:51493,CHEBI:51494,CHEBI:51504
id,,,,,
urn:uuid:6c7238d0-5836-4cfe-be27-63837059086b,NaN,CHEBI:26547,NaN,NaN,NaN
urn:uuid:875773d1-301d-46e4-bc42-1156d7cfaafa,NaN,NaN,NaN,CHEBI:51503,NaN
urn:uuid:a2ef1a16-b470-4b35-a6c9-0a2681b3fe2f,NaN,NaN,NaN,NaN,CHEBI:51505
urn:uuid:eebedf42-5961-4208-aaae-44796c6574e6,CHEBI:26548,NaN,NaN,NaN,NaN
urn:uuid:f64ba0ef-1152-40b5-b245-bed32d57443f,NaN,NaN,CHEBI:51504,NaN,NaN


## approach 2: create a test dataframe using subject values that occur more than 20 times

In [9]:
testDf = df.groupby('subject').filter(lambda x: len(x) > 20)
len(testDf)

120

In [10]:
testDf.subject.value_counts()

CHEBI:3962       30
NCBITaxon:287    25
CHEBI:16118      23
CHEBI:4775       21
NCBITaxon:562    21
Name: subject, dtype: int64

In [11]:
testDf.head() # peek at data

,id,subject,predicate,object,relation,provided_by
145309,urn:uuid:ef61c57d-1fc2-44c2-bb5d-c5d9afe8d0ee,CHEBI:3962,biolink:has_attribute,CHEBI:53746,RO:0000087,chebi
145310,urn:uuid:fe86c705-cbe7-4ccd-8225-443493c55864,CHEBI:3962,biolink:has_attribute,CHEBI:64670,RO:0000087,chebi
145311,urn:uuid:030b4615-f2ed-432d-889a-3b021eff2c1b,CHEBI:3962,biolink:has_attribute,CHEBI:26130,RO:0000087,chebi
145312,urn:uuid:fa519ca3-a590-4773-99ca-452c59eccf56,CHEBI:3962,biolink:has_attribute,CHEBI:37958,RO:0000087,chebi
145313,urn:uuid:73e4d0e2-8eb8-48ff-84bc-77f7861f04f4,CHEBI:3962,biolink:has_attribute,CHEBI:48550,RO:0000087,chebi


### pivot `testdf` so that that the subject values are values in each object column

In [18]:
pivotTestDf = testDf.pivot(index='subject', columns='object', values='subject')
pivotTestDf

object,CHEBI:131699,CHEBI:17620,CHEBI:22586,CHEBI:22754,CHEBI:24632,CHEBI:25000,CHEBI:25212,CHEBI:26130,CHEBI:26195,CHEBI:30778,...,NCBITaxon:1,NCBITaxon:136841,NCBITaxon:561,PATO:0001472,Shape:bacillus,Shape:coccobacillus,UBERON:0000065,UBERON:0000178,UBERON:0000468,UBERON:0000996
subject,,,,,,,,,,,,,,,,,,,,,
CHEBI:16118,NaN,NaN,CHEBI:16118,CHEBI:16118,NaN,NaN,CHEBI:16118,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CHEBI:3962,NaN,CHEBI:3962,NaN,NaN,NaN,NaN,CHEBI:3962,CHEBI:3962,CHEBI:3962,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CHEBI:4775,CHEBI:4775,NaN,CHEBI:4775,NaN,NaN,CHEBI:4775,NaN,NaN,CHEBI:4775,CHEBI:4775,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NCBITaxon:287,NaN,NaN,NaN,NaN,NCBITaxon:287,NaN,NaN,NaN,NaN,NaN,...,NCBITaxon:287,NCBITaxon:287,NaN,NCBITaxon:287,NCBITaxon:287,NaN,NCBITaxon:287,NCBITaxon:287,NCBITaxon:287,NaN
NCBITaxon:562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NCBITaxon:562,NaN,NCBITaxon:562,NCBITaxon:562,NCBITaxon:562,NCBITaxon:562,NCBITaxon:562,NCBITaxon:562


### validate output from test dataframe
Steps
- get the subject and object from `testDf` where the index is `NCBITaxon:562`; store this result in `testResult`
- get the subject and object from the pivot dataframe; store this result in `pivotResult`
- perform a left outer join on `pivotResult` and `testResult`; the result is valid if:
  * every result from the `pivotResult['NCBITaxon:562']` dataframe has the same value in the `testResult['subject']` column
  * every NaN from `pivotResult['NCBITaxon:562']` is also NaN in `testResult['subject']`

In [30]:
testResult = testDf[testDf.subject == 'NCBITaxon:562'][['subject', 'object']]
# testResult
print('(rows, cols):', testResult.shape)
testResult

(rows, cols): (21, 2)


,subject,object
288097,NCBITaxon:562,ECOCORE:00000180
288098,NCBITaxon:562,ECOCORE:00000177
288099,NCBITaxon:562,Shape:bacillus
288100,NCBITaxon:562,UBERON:0000468
288101,NCBITaxon:562,ENVO:2100002
288102,NCBITaxon:562,UBERON:0000996
288103,NCBITaxon:562,ENVO:2100003
288104,NCBITaxon:562,UBERON:0000065
288105,NCBITaxon:562,ENVO:00002149
288106,NCBITaxon:562,ENVO:00002123


In [31]:
# pds.set_option('display.max_rows', 500) # uncomment to see all of result
pivotResult = pivotTestDf.loc['NCBITaxon:562'].reset_index()
print('(rows, cols):', pivotResult.shape)
pivotResult

(rows, cols): (99, 2)


,object,NCBITaxon:562
0,CHEBI:131699,NaN
1,CHEBI:17620,NaN
2,CHEBI:22586,NaN
3,CHEBI:22754,NaN
4,CHEBI:24632,NaN
...,...,...
94,Shape:coccobacillus,NCBITaxon:562
95,UBERON:0000065,NCBITaxon:562
96,UBERON:0000178,NCBITaxon:562
97,UBERON:0000468,NCBITaxon:562


visually inspecting the output, we see that the values and NaNs line up as expected.

In [34]:
pds.set_option('display.max_rows', 500)
pds.merge(pivotResult, testResult, how='left', on='object')

,object,NCBITaxon:562,subject
0,CHEBI:131699,NaN,NaN
1,CHEBI:17620,NaN,NaN
2,CHEBI:22586,NaN,NaN
3,CHEBI:22754,NaN,NaN
4,CHEBI:24632,NaN,NaN
5,CHEBI:25000,NaN,NaN
6,CHEBI:25212,NaN,NaN
7,CHEBI:26130,NaN,NaN
8,CHEBI:26195,NaN,NaN
9,CHEBI:30778,NaN,NaN
